In [1]:
# Imports
import cobra
from cobra.util.solver import linear_reaction_coefficients
import gurobipy
import cobra.util.array as cba
import pandas as pd
# STops the truncation of the results with "...""
pd.options.display.max_rows = 4000

import numpy as np

# for writing out files without overwriting other ones
from datetime import date
import os
import escher

# files for loopless and pFBA
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba

# files for FVA
from cobra.flux_analysis import flux_variability_analysis

# for writing out excel files
import xlsxwriter

import datetime
from os import path

import math

In [2]:
def rxnBalanced(model):
    """if positive H:1 and charge:1 then you add a H+ to the reactant side, if H:-1 and charge: -1
    then you add the H+ to the product side"""

    l = len(model.reactions)
    counter = 0

    for x in range(l):
        rxn_id = model.reactions[x].id
        #  The dictionary with the mass balance
        mbD = model.reactions.get_by_id(rxn_id).check_mass_balance()

        if bool(mbD) == True:
            counter += 1

            print("The mass and/or charge is not balanced in rxn", rxn_id)
            print("Balance: ", mbD)
    print("Number of unbalanced reactions is: ", counter)

In [3]:
# function to print matrix as table

def Smatrix2table(model, matrix):
    '''this function returns a matrix written as a table with labels'''
    shape = matrix.shape
    nummetabolites = shape[0]
    numreactions = shape[1]
    listMet =[]
    listRxn = []
    for m in range(len(model.metabolites)):
        metid = model.metabolites[m].id
        listMet.append(metid)
    for r in range(len(model.reactions)):
        rxnid = model.reactions[r].id
        listRxn.append(rxnid)    
    table = pd.DataFrame(matrix,index=listMet,columns=listRxn)
    return table

In [4]:
# function to print out outputs of optimization in a nice looking DF, returns an alphabetized table and a list to put in the original order

def solution_as_table(model, solution):
    
    #series
    OV = solution.objective_value
    SS = solution.status
    FX = solution.fluxes
    SP = solution.shadow_prices
    RC = solution.reduced_costs
    
    #dict
    FX_d = solution.fluxes.to_dict()
    SP_d = solution.shadow_prices.to_dict()
    RC_d = solution.reduced_costs.to_dict()
    
    #dataframe
    FX_df = pd.DataFrame.from_dict(FX_d,orient = "index")
    SP_df = pd.DataFrame.from_dict(SP_d,orient = "index")
    RC_df = pd.DataFrame.from_dict(RC_d,orient = "index")
    
    #initialize dataframe
    frames = [FX_df,RC_df,SP_df]
    df = pd.concat(frames,axis =1)#,keys=['Reactions','Reactions','Metabolites'])
    df.columns = ['Fluxes','Reduced Costs','Shadow Prices']
    
    #makes the order in a list format
    orderrxn = list(FX_d.keys())
    ordermet = list(SP_d.keys())
    order = orderrxn+ordermet
    
    #prints values outside of the dataframe
    print ("Objective Value:\t",OV)
    print ("Solution Status:\t",SS)
    return df, order

In [5]:
# function to print out outputs of optimization in a nice looking DF, returns an alphabetized table and a list to put in the original order

def solution_as_table_short(model, solution):
    
    #series
    OV = solution.objective_value
    SS = solution.status
    FX = solution.fluxes

    #dict
    FX_d = solution.fluxes.to_dict()

    #dataframe
    FX_df = pd.DataFrame.from_dict(FX_d,orient = "index")

    #initialize dataframe
    frames = [FX_df]#,RC_df,SP_df]
    df = pd.concat(frames,axis =1)#,keys=['Reactions','Reactions','Metabolites'])
    df.columns = ['Fluxes']#,'Reduced Costs','Shadow Prices']
    
    #makes the order in a list format
    orderrxn = list(FX_d.keys())
    order = orderrxn#+ordermet
    
    #prints values outside of the dataframe
    print ("Objective Value:\t",OV)
    print ("Solution Status:\t",SS)
    return df, order

In [6]:
def nonzero_flux(model,solution):

    #dict
    FX_d = solution.fluxes.to_dict()
    
    for key, value in FX_d.items() :
        if value != 0.0:
            rxn = model.reactions.get_by_id(key).name
            print ("\033[1m",key, ":","\033[0m",rxn, ":", value)

In [7]:
def makeMap(directory, mapname, model, solution):
    os.chdir(directory)
    print(os.getcwd())
    
    builder = escher.Builder(model=model)
    reactiondata = solution.fluxes
    index = reactiondata.index

    reactiondict = {}
    for i in range(len(reactiondata)):
        key = reactiondata.index[i]
        value = reactiondata.values[i]
        valueint = float(value)
        reactiondict[key] = valueint
    
        #set values to the new 
    print(reactiondict)
    b = escher.Builder(map_json=mapname,
                   reaction_data=reactiondict,
                   # change the default colors
                   #reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                    #               {'type': 'mean', 'color': '#0000dd', 'size': 20},
                     #              {'type': 'max', 'color': '#ff0000', 'size': 40}],
                   # only show the primary metabolites
                   hide_secondary_metabolites=False)
    return b   

In [85]:
# path to the model files
# path2model = r"F:\GEM_Models\Final_Model\Final_Publish\20220623_DH_model.xml"
path2model = r"F:\GEM_Models\Final_Model\Final_Publish\20220623_ZZ_model.xml"
# path2model = r"F:\GEM_Models\Final_Model\Final_Publish\20220623_MM_model.xml"

In [86]:
#  Reads in the model
model = cobra.io.read_sbml_model(path2model)
model

Name,iMTZ22IC
Memory address,0x01d84880e080
Number of metabolites,555
Number of reactions,619
Number of groups,104
Objective expression,1.0*EX_biomass_e - 1.0*EX_biomass_e_reverse_5e4c9
Compartments,"cytosol, extracellular"


In [87]:
print("Number of genes in the model: ",len(model.genes))

Number of genes in the model:  520


## Validate model

In [88]:
cobra.io.sbml.validate_sbml_model(path2model, check_model=True) # Use with the newer COBRA

(<Model iMTZ22IC at 0x1d84a9964e0>,
 {'SBML_FATAL': [],
  'SBML_ERROR': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': [],
  'COBRA_FATAL': [],
  'COBRA_ERROR': [],
  'COBRA_WARNING': [],
  'COBRA_CHECK': []})

## Set solver

In [89]:
model.solver = "gurobi"
print("MODEL OBJECTIVE: \n",model.objective)
print("MODEL SOLVER: \n",model.solver)

MODEL OBJECTIVE: 
 Maximize
1.0*EX_biomass_e - 1.0*EX_biomass_e_reverse_5e4c9
MODEL SOLVER: 
 \ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  EX_biomass_e - EX_biomass_e_reverse_5e4c9
Subject To
 h2_c: - 2 MVHHDR + 2 MVHHDR_reverse_5da06 - FRH + FRH_reverse_2a109 - HMD
   + HMD_reverse_23491 + 2 CARS - 2 CARS_reverse_24ade + 2 GCARS
   - 2 GCARS_reverse_e0ab6 + 2 2GCARS - 2 2GCARS_reverse_c9167 + 2 PECARS
   - 2 PECARS_reverse_1b82c + 2 2PECARS - 2 2PECARS_reverse_18dce
   + 2 PICARS - 2 PICARS_reverse_1a275 + 2 2GPICARS
   - 2 2GPICARS_reverse_e6c45 + GPECARS - GPECARS_reverse_5eced + 2GPECARS
   - 2GPECARS_reverse_eeff2 + NIT1b2 - NIT1b2_reverse_497d5 - Eha
   + Eha_reverse_5bd35 - Ehb + Ehb_reverse_822cd + tH2 - tH2_reverse_484e3
   = 0
 fdxox_c: - MVHHDR + MVHHDR_reverse_5da06 + FWD - FWD_reverse_c9261 + CbiL
   - CbiL_reverse_f0d0a + 3 HTCAF - 3 HTCAF_reverse_9de33 + CoMF
   - CoMF_reverse_5047b + CODH2 - CODH2_reverse_44594 + CODHr2
   - 




## Checks For Rxn Balance

In [90]:
# Values lower than zero indicate missing atoms on the product side 
#whereas positive values indicate missing atoms on the substrate side
# to add a metabolite to the substrate side, the coefficient should be less than zero
# to add a metabolite to the product side, the coefficient should be greater than zero
rxnBalanced(model)

The mass and/or charge is not balanced in rxn PROTEIN_Z
Balance:  {'charge': -0.07839999999999492, 'C': -44.38330000000002, 'H': -88.98930000000001, 'N': -12.419299999999993, 'O': -22.22220000000013, 'S': -0.38570000000000004, 'P': -7.105427357601002e-15}
The mass and/or charge is not balanced in rxn DNA_Z
Balance:  {'C': -31.690399999999997, 'H': -39.8139, 'N': -12.176499999999997, 'O': -19.496399999999998, 'P': -3.2494000000000005}
The mass and/or charge is not balanced in rxn RNA_Z
Balance:  {'charge': -1.5543122344752192e-15, 'C': -29.621599999999994, 'H': -36.52440000000001, 'N': -11.8732, 'O': -21.78529999999999, 'P': -3.107199999999998}
The mass and/or charge is not balanced in rxn SMP_Z
Balance:  {'charge': -4.440899999999999, 'C': -26.574799999999996, 'H': -35.9709, 'N': -6.7433, 'Ni': -0.24509999999999998, 'O': -15.1221, 'P': -1.3828999999999998, 'S': -0.9665000000000001, 'Fe': -0.5281, 'Co': -0.489, 'Zn': -0.4247, 'Mg': -1.1429, 'Ca': -0.6931, 'K': -0.7105, 'Mo': -0.20579999

# Part 1: Prepare model for general use

# Turn off multiple option pathways (stylistic)

In [91]:
# reaction written together/apart (stylistic)
model.reactions.get_by_id("HISTDa").bounds = (0,0)
model.reactions.get_by_id("HISTDb").bounds = (0,0)
# model.reactions.get_by_id("HISTD").bounds = (0,0) # combined version

# model.reactions.CODH2.bounds = (0,0) # required to be on for growth on CO
# model.reactions.ACS3.bounds = (0,0) # required to be on for growth on CO
model.reactions.CODHr2.bounds = (0,0) # combined version (doesn't work for growth on CO)

# alternative to PFL, but accounting for activating enzymes (technically only present in DH)
model.reactions.get_by_id("PFL2").bounds = (0,0)
# model.reactions.PIt2r.bounds = (0,0)

# Make model reaction wise microbe specific

In [92]:
# For ALL
model.reactions.Ehb.bounds = (0,0) # want only one active (Eha)
# model.reactions.get_by_id("HMD").bounds = (-1000,1000) #(not sure if it should be zero) Also prevents loops
# model.reactions.get_by_id("MTD").bounds = (-1000,1000) # Also prevents loops

# model.reactions.FRH.bounds = (0,1000)
#--------------------------------------------------------------------------------------------------------------
# Make bounds (0,0) for DH and ZZ

# Methanopepterin Biosynthesis (methylation at different point)
# The following reactions that are turned off are for the Marburgensis, so when using MM we can comment them out
model.reactions.get_by_id("DHPS3").bounds = (0,0) #(dhrfapF equivalent for Marburgensis)
model.reactions.get_by_id("H4MPTS9").bounds = (0,0) # (dhadrpF equivalent for Marburgensis)
model.reactions.get_by_id("dhadrF2").bounds = (0,0) # (dhadrF equivalent for Marburgensis)

# Psuedomurein Biosynthesis (Galactosamine -Marburg vs glucosamine at different point)
# The following reactions that are turned off are for the Z-245 and DH
model.reactions.get_by_id("UGALNACS").bounds = (0,0) # UACNACS equivalent for Marburgensis
model.reactions.get_by_id("PSMNS2").bounds = (0,0) # PSMNS equivalent for Marburgensis

#---------------------------------------------------------------------------------------------------------------
# Make bounds (0,0) for MM

# # Methanopepterin Biosynthesis (methylation at different point)
# # The following reactions that are turned off are for the Z-245 and DH, so when using DH/ZZ we can comment them out
# model.reactions.get_by_id("dhrfapF").bounds = (0,0) #(DHPS3 equivalent for DH/Z-245)
# model.reactions.get_by_id("dhadrpF").bounds = (0,0) # (H4MPTS9 equivalent for DH/Z-245)
# model.reactions.get_by_id("dhadrF").bounds = (0,0) # (dhadrF2 equivalent for DH/Z-245)

# # Psuedomurein Biosynthesis (Galactosamine -Marburg vs glucosamine at different point)
# # The following reactions that are turned off are for the Marburgensis
# model.reactions.get_by_id("UACNACS").bounds = (0,0) # UGALNACS equivalent for DH/Z-245
# model.reactions.get_by_id("PSMNS").bounds = (0,0) # PSMNS2 equivalent for DH/Z-245

# # homologus genes not found for MM
# model.reactions.get_by_id("RNDR1").bounds = (0,0) # no gene in Marburg?
# model.reactions.get_by_id("RNDR2").bounds = (0,0) # no gene in Marburg?
# model.reactions.get_by_id("RNDR3").bounds = (0,0) # no gene in Marburg?
# model.reactions.get_by_id("RNDR4").bounds = (0,0) # no gene in Marburg?
# model.reactions.PFL.bounds = (0,0) # only in DH (0,1000) otherwise makes loop
# model.reactions.PFL2.bounds = (0,0) # only in DH (0,1000) otherwise makes loop
#-----------------------------------------------------------------------------

# Make bounds (0,0) for ZZ
model.reactions.PFL.bounds = (0,0) # only in DH (-1000,1000) otherwise makes loop
model.reactions.PFL2.bounds = (0,0) # only in DH (-1000,1000) if not using PFL otherwise makes loop

# Make bounds (0,0) for DH/MM and ZZ not grown on formate

# Energy Metabolism
model.reactions.get_by_id("FDH_F420").bounds = (-1000,1000) # (0,1000) ONLY Z-245, growth of formate (if added to media, evidence only Z-245)

# Define standard media with bounds differences by microbe
Doesn't include energy sources

In [93]:
# For all microbes make the bounds (0,0) unless added to media
model.reactions.get_by_id("EX_4abz_e").bounds = (0,0) # this can be used as alternative to 4hbz_c (BRAHPS) (if added to media)
model.reactions.get_by_id("EX_4hphac_e").bounds = (0,0) # assimiliation of 4-Hydroxyphenylacetate (if added to media, in maripaludis)
model.reactions.get_by_id("EX_btn_e").bounds = (0,0) # assimiliation biotin (if added to media)
model.reactions.get_by_id("EX_ind3ac_e").bounds = (0,0) # assimiliation of Indole-3-acetate (if added to media, in maripaludis)

# DH and ZZ can never use, but MM can when provided (not standard media)
model.reactions.get_by_id("EX_ppa_e").bounds = (0,0) # ONLY MARBURGENSIS - exchange of propanoate (if added to media)
model.reactions.get_by_id("EX_pyr_e").bounds = (0,0) # ONLY MARBURGENSIS - assimiliation of pyruvate (if added to media, only by Marburgensis)
model.reactions.get_by_id("EX_5aop_e").bounds = (0,0) # ONLY MARBURGENSIS - assimiliation of 5 Aminolevulinate (if added to media, only by Marburgensis)
model.reactions.get_by_id("EX_fum_e").bounds = (0,0) # ONLY MARBURGENSIS - assimiliation of fumarate (if added to media, only by Marburgensis)
model.reactions.get_by_id("EX_succ_e").bounds = (0,0) # ONLY MARBURGENSIS - assimiliation of succinate (if added to media, only by Marburgensis)
model.reactions.get_by_id("EX_pac_e").bounds = (0,0) # ONLY MARBURGENSIS - assimiliation of phenylacetate (if added to media, evidence only in Marburgensis)
model.reactions.get_by_id("EX_pro__L_e").bounds = (0,0) # none added to the model and none measured...
model.reactions.get_by_id("EX_ac_e").bounds = (0,0) # MARBURGENSIS and maybe Z-245 - assimiliation of acetate (if added to media, only in Marburgensis, and Z-245)

# Growth constrained Energy/Carbon/Sulfur sources

In [110]:
# Energy (electron) sources
model.reactions.get_by_id("EX_h2_e").bounds = (0,1000)
# Carbon sources
model.reactions.get_by_id("EX_co2_e").bounds = (0,1000)

# Energy + Carbon souces
#Formate requires the FDH_F420 (0,1000) and for CO2 to be let out
model.reactions.get_by_id("EX_for_e").bounds = (-1000,0) # ONLY Z-245 (or Marburgensis for assimiliation/growth of formate (if added to media, evidence only in Marburgensis (assimilation) and Z-245 (growth))
model.reactions.get_by_id("EX_co_e").bounds = (0,0) # assimiliation carbon monoxide (if added to gas, only by Marburgensis)
# for growth on CO
model.reactions.Eha.bounds = (0,1000) #(-1000,1000) for CO otherwise (0,1000)

# Sulfur Sources
model.reactions.get_by_id("EX_cys__L_e").bounds = (0,1000) # -0.0145 assimiliation of cysteine (H2S must be left open?)
model.reactions.get_by_id("EX_h2s_e").bounds = (-1000,1000) # 

# Nitrogen Sources
model.reactions.get_by_id("EX_nh3_e").bounds = (-1000,1000) # not quantifiable
model.reactions.get_by_id("EX_n2_e").bounds = (0,0) # assimiliation of nitrogen (if added to gas)

# Limit Products

In [111]:
model.reactions.get_by_id("EX_ch4_e").bounds = (0,1000)
model.reactions.get_by_id("EX_biomass_e").bounds = (0,1000)


# Initalize ATPM reaction

In [112]:
model.reactions.get_by_id("ATPM").bounds = (1.5,1000)

# End part 1 with general constraints
Using the constraints provided above, the model should be able to predict general phenotypes
It will consume lots of cysteine if it is not constrained.

## FBA from part 1

In [113]:
# Define objective function
obj = {model.reactions.get_by_id("EX_biomass_e"):1}
# obj = {model.reactions.get_by_id("ATPM"):1}
model.objective = obj

In [114]:
solution = model.optimize()
print(solution)

<Solution 1.383 at 0x1d84aab7208>


In [115]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
2dglc_e,EX_2dglc_e,0.001869,6,0.00%
ca2_e,EX_ca2_e,0.008624,0,0.00%
cobalt2_e,EX_cobalt2_e,0.006085,0,0.00%
fe2_e,EX_fe2_e,0.006571,0,0.00%
for_e,EX_for_e,1000,1,100.00%
h2s_e,EX_h2s_e,0.3974,0,0.00%
h_e,EX_h_e,985,0,0.00%
k_e,EX_k_e,0.008841,0,0.00%
mg2_e,EX_mg2_e,0.01422,0,0.00%
mobd_e,EX_mobd_e,0.002561,0,0.00%


In [116]:
solution = solution.to_frame()
solution

,fluxes,reduced_costs
MVHHDR,221.151011,-2.081668e-17
FRH,-738.113988,-0.000000e+00
HMD,237.913980,1.110223e-16
FTRM,237.913980,-1.110223e-16
FWD,237.913980,0.000000e+00
MCH,237.913980,0.000000e+00
MTD,0.000000,-2.220446e-16
MER,239.701819,-1.110223e-16
MTR,218.860720,-3.469447e-17
MCR,218.860720,0.000000e+00


In [117]:
# with open(r"F:\GEM_Models\Final_Model\Final_Publish\FBA_Check\FBA_ZZ_for_h2s.txt", "w") as outfile:
#     outfile.write(str(solution))
#     outfile.close()

In [86]:
# cobra.io.write_sbml_model(model, r"F:\GEM_Models\Final_Model\Final_Final\20220203_DH_model_bound.xml")

In [ ]:
# cobra.io.save_matlab_model(model, r"F:\GEM_Models\Final_Model\Final_Final\20220203_DH_model_bound.mat")

## Display in Map

In [ ]:
path = r"F:\GEM_Models\Final_Model"
mapnormal = makeMap(path,"DH_map_final_final.json",model,solution)
# mapnormal = makeMap(path,"ZZ_map_final_final.json",model,solution)
# mapnormal = makeMap(path,"MM_map_final_final.json",model,solution)

mapnormal.display_in_browser()#notebook()#browser()

## Export to Text File

In [ ]:
# with open(r"F:\GEM_Models\Final_Model\FBA_Check2\FBA_ZZ_for_cys.txt", "w") as outfile:
#     outfile.write(str(solution))
#     outfile.close()

## Run pFBA

In [ ]:
solutionpfba = pfba(model)
solutionpfba = solutionpfba.to_frame()
solutionpfba

In [ ]:
solutionloopless = loopless_solution(model)
solutionloopless = solutionloopless.to_frame()
solutionloopless

In [ ]:
path = r"F:\GEM_Models\Final_Model"
# mapnormal = makeMap(path,"DH_map_final_final.json",model,solutionpfba)
mapnormal = makeMap(path,"DH_map_final_final.json",model,solutionloopless)

# mapnormal = makeMap(path,"ZZ_gene_map.json",model,solution)
# mapnormal = makeMap(path,"MM_gene_map.json",model,solution)

mapnormal.display_in_browser()#notebook()#browser()

In [ ]:
round(solution,1) == round(solutionpfba,1)

# Part2: FVA to show lack of metabolic flexibility 

In [74]:
obj = {model.reactions.get_by_id("EX_biomass_e"):1}
model.objective = obj

In [75]:
# model.optimize()
# model.summary(fva=0.95)

In [76]:
solutionfva = cobra.flux_analysis.flux_variability_analysis(
    model, fraction_of_optimum=0.95)
print(solutionfva)

                  minimum      maximum
MVHHDR         210.252833   222.605552
FRH           -524.888237  1000.000000
HMD           -524.888237  1000.000000
FTRM           226.166659   238.519378
FWD            226.166659   238.519378
MCH            226.166659   238.519378
MTD           -773.833341   751.054897
MER            227.865481   240.218200
MTR            208.076578   220.429297
MCR            208.076578   220.429297
NAt3_1         -14.988839   -10.946064
NAATP           72.985351    79.277529
CBPS             0.614704     0.647057
MoaA             0.001217     0.001281
MoaC             0.001217     0.001281
MoaE             0.001217     0.001281
MPTAT            0.000380     0.000399
MoeAB            0.000380     0.000399
MobAB            0.000380     0.000399
GLUTRS           0.007482     0.007876
GLUTRR           0.007482     0.007876
G1SAT            0.007482     0.007876
PPBNGS           0.003741     0.003938
HMBS             0.000935     0.000984
UPP3S            0.000935

In [77]:
# with open(r"F:\GEM_Models\Final_Model\Final_Publish\FBA_Check\FVA_ZZ_h2co2_h2s_cys.txt", "w") as outfile:
#     outfile.write(str(solutionfva))
#     outfile.close()